## Seleção de Features

Neste tópico abordaremos a seleção de features, utilizando o método Boruta, teste de Chi2 e Correlação point-biserial para tal.


Teste de correlação ponto-bisserial é simplesmente a correlação entre uma variável dicotômica e uma variável conntínua. Acontece que este é um caso especial da correlação de Pearson. Portanto, calcular a correlação bisserial de ponto especial é equivalente a calcular a correlação de Pearson quando uma variável é dicotômica e a outra é contínua.

Chi2: Método utilizado para seleção de features categóricas ordinais e nominais. É um método que avalia a indepedência das features em relação ao target.

Boruta: Método muito robusto e baseado em modelo, que leva em consideração a interação entre a features, algo nao levado em conta pelo Chi2 ou outros métodos univariados de seleção. 

Para mais informações sobre o Boruta: https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a

## Depêndencias

In [1]:
#Importando bibliotecas
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cloudpickle
import matplotlib.pyplot as plt
from scipy.stats import norm
from pandas.api.types import is_numeric_dtype
from utils.selecao_de_features import boruta_selector,chi_squared,point_biserial
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline

## Dados e Constantes

In [2]:
# Constantes
DATA_INTER_PATH = '../data/inter'
TRAIN_DATA = 'diabetes_train.parquet'
TRAIN_DATA_SELECTED = 'diabetes_train_selected.parquet'
# Dados
df_train = pd.read_parquet(os.path.join(DATA_INTER_PATH, TRAIN_DATA))

In [3]:
df_train.head()

,idade,genero,polyuria,polydipsia,perda_de_peso_repentina,fraqueza,polyphagia,candidiase_genital,desfoque_visual,coceira,irritabilidade,cicatrizacao_retardada,paresia_parcial,rigidez_muscular,alopecia,obesidade,target
0,39,female,yes,yes,yes,yes,yes,no,no,yes,yes,yes,yes,no,no,no,1
1,62,female,yes,yes,yes,yes,no,no,yes,no,no,no,yes,no,no,yes,1
2,36,male,no,no,no,no,no,no,no,no,no,no,no,no,no,no,0
3,30,male,no,no,no,no,no,no,no,no,no,no,no,no,no,no,0
4,38,female,yes,yes,yes,yes,yes,no,no,no,no,no,yes,no,no,no,1


In [4]:
# separando variaveis numericas e categóricas
todas_as_variaveis = set(df_train.columns.tolist())
variaveis_categoricas = set(df_train.select_dtypes(include=['object']).columns.tolist())
variaveis_numericas = todas_as_variaveis - variaveis_categoricas
variaveis_categoricas = list(variaveis_categoricas)

In [5]:
# Convertendo os daods para numérico
df_train = df_train.replace({'female':0, 'male':1, 'no':0, 'yes':1})
# Criando cópia do dataFrame
df_train_copy = df_train.copy()

# Abordagem (Correlação+Chi2+Boruta)

## Correlação

In [6]:
pb_df, columns_remove_pb = point_biserial(df_train, 'target', ['idade'])

In [7]:
pb_df

,column,correlation,p_value,result
0,idade,0.140436,0.007287,Reject H0


In [8]:
columns_remove_pb

[]

O test point biserial não eliminou a feature idade.

## Chi2

In [9]:
chi2_df, logs = chi_squared(df_train, y ='target', cols = variaveis_categoricas)

In [10]:
chi2_df

,column,p-value,chi2_value
0,obesidade,1.679975e-02,5.717174
1,fraqueza,3.135309e-05,17.333975
2,desfoque_visual,1.340147e-06,23.364710
3,alopecia,9.635603e-07,23.999594
4,polyphagia,2.205976e-12,49.291667
5,perda_de_peso_repentina,8.868902e-18,73.749449
6,cicatrizacao_retardada,3.471569e-01,0.883822
7,paresia_parcial,6.173565e-20,83.562417
8,polydipsia,1.312510e-36,159.705000
9,coceira,1.000000e+00,0.000000


In [11]:
logs

[]

Com é o próprio target, não calculamos o chi2 para ele.
* Hipótese nula - H0: As variáveis são independente. 
- Segundo o teste de hipotése realizado, as variáveis cujo p-valor < 0.05 tem relação com o target.

In [12]:
#As variáveis que serão excluídas. 
lista_descart_chi2 = chi2_df[chi2_df['p-value']>=0.05]['column'].tolist()

In [13]:
lista_descart_chi2

['cicatrizacao_retardada', 'coceira']

As variáveis que foram descartas pelo teste de Chi-2 foram, `cicatrização_retardada` e `coceira`, logo mais elas serão descartadas para a modelagem. 

## Boruta

In [14]:
#Vamos filtrar as Features removidas no step anterior antes de passarmos para o Boruta
list_to_drop_boruta = boruta_selector(df_train_copy.drop(lista_descart_chi2,axis=1), y='target')

## Seleção Geral

In [15]:
#Lista de dropagem Geral (chi2 + boruta)
lista_drop_geral = set(lista_descart_chi2+list_to_drop_boruta+columns_remove_pb)
lista_drop_geral

{'alopecia',
 'candidiase_genital',
 'cicatrizacao_retardada',
 'coceira',
 'desfoque_visual',
 'fraqueza',
 'irritabilidade',
 'obesidade',
 'rigidez_muscular'}

# Exportando dataset

In [16]:
# dropando as colunas pela abordagem.
df_train_copy.drop(lista_drop_geral,axis=1).to_parquet(os.path.join(DATA_INTER_PATH,TRAIN_DATA_SELECTED), index=False)

A abordagem para seleção de features que utilizamos são (Chi2 e Boruta), que combinados trazem resultados bastante relevantes, descartando as variáveis que possuem menos relação com target e que consequentemente teriam um baixo poder preditivo. As features descartadas pelos métodos foram:`candidíase_genital`, `cicatrização_retardada`, `coceira`, `desfoque_visual`, `fraqueza`, `obesidade`, `regidez_muscular`, `alopecia`, `irritabilidade` . Todas essas features não serão consideradas para nosso modelo.